<a href="https://colab.research.google.com/github/badBrock/MultiHeadAttentionJourney/blob/main/Causal_Self_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

In [ ]:
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [ ]:
input_query = inputs[1]
input_query

tensor([0.5500, 0.8700, 0.6600])

In [ ]:
input_query @ inputs.T

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])

In [ ]:
attn_score2 = torch.empty(inputs.shape[0])

In [ ]:
for i in range(len(inputs)):
  attn_score2[i] = torch.dot(input_query,inputs[i])

In [ ]:
attn_score2

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])

In [ ]:
attn_weights = torch.softmax(attn_score2,dim=0)
attn_weights

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

In [ ]:
attn_weights_2 = attn_weights

In [ ]:
cxt_vector = torch.zeros(inputs.shape[1])
for i in range(len(inputs)):
  cxt_vector += attn_weights[i] * inputs[i]

In [ ]:
(cxt_vector)

tensor([0.4419, 0.6515, 0.5683])

In [ ]:
#! now we can do this for all the values here

In [ ]:
# we have inputs
inputs

tensor([[0.4300, 0.1500, 0.8900],
        [0.5500, 0.8700, 0.6600],
        [0.5700, 0.8500, 0.6400],
        [0.2200, 0.5800, 0.3300],
        [0.7700, 0.2500, 0.1000],
        [0.0500, 0.8000, 0.5500]])

In [ ]:
# to get the attn_score we do

all_attn_scores = inputs @ inputs.T
all_attn_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [ ]:
all_attn_weights = torch.softmax(all_attn_scores,dim=1)
all_attn_weights

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [ ]:
for i in range(len(all_attn_weights)):
  print(all_attn_weights.sum(axis=1))

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])
tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])
tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])
tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])
tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])
tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [ ]:
all_context = all_attn_weights @ inputs

In [ ]:
all_context

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

In [ ]:
import torch.nn as nn

class Attention(nn.Module):
  def __init__(self,d_in,d_out):
    super().__init__()
    # inputs dim is what his d_in is, i.e the dim we already have for our inputs
    self.W_q = torch.nn.Linear(d_in,d_out,bias=False)
    self.W_k = torch.nn.Linear(d_in,d_out,bias=False)
    self.W_v = torch.nn.Linear(d_in,d_out,bias=False)

  def forward(self,x):
    self.Q = self.W_q(x)
    self.K = self.W_q(x)
    self.V = self.W_q(x)

    attn_score = self.Q @ self.K.T
    attn_weights = torch.softmax(attn_score,dim=1)

    context_vec = attn_weights @ self.V

    return context_vec


In [ ]:
inputs.shape

torch.Size([6, 3])

In [ ]:
call = Attention(3,1)

In [ ]:
ctx = call(inputs)

In [ ]:
ctx

tensor([[0.1668],
        [0.1534],
        [0.1537],
        [0.1479],
        [0.1582],
        [0.1451]], grad_fn=<MmBackward0>)

In [ ]:
temp = ctx[:5,:5]

In [ ]:
ctx.shape

torch.Size([6, 6])

In [ ]:
mask = torch.triu(torch.ones(ctx.shape[0], len(ctx)),diagonal=1)
mask

tensor([[0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])

In [ ]:
print("ctx shape :", ctx.shape)
print("mask shape:", mask.shape)


ctx shape : torch.Size([6, 6])
mask shape: torch.Size([6, 6])


In [ ]:
masked = ctx.masked_fill(mask.bool(),-torch.inf)
masked

tensor([[ 0.0320,    -inf,    -inf,    -inf,    -inf,    -inf],
        [ 0.0463,  0.1210,    -inf,    -inf,    -inf,    -inf],
        [ 0.0465,  0.1203, -0.5353,    -inf,    -inf,    -inf],
        [ 0.0410,  0.1110, -0.5072,  0.7491,    -inf,    -inf],
        [ 0.0498,  0.0966, -0.5066,  0.7450,  0.4531,    -inf],
        [ 0.0382,  0.1200, -0.5150,  0.7614,  0.4621,  0.7444]],
       grad_fn=<MaskedFillBackward0>)

In [ ]:
attn_weights = torch.softmax(masked,dim=-1)

In [ ]:
attn_weights

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4814, 0.5186, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3795, 0.4085, 0.2121, 0.0000, 0.0000, 0.0000],
        [0.2137, 0.2291, 0.1235, 0.4337, 0.0000, 0.0000],
        [0.1633, 0.1712, 0.0936, 0.3274, 0.2445, 0.0000],
        [0.1208, 0.1311, 0.0695, 0.2491, 0.1846, 0.2449]],
       grad_fn=<SoftmaxBackward0>)

In [ ]:
batch = torch.stack((inputs,inputs), dim = 0)
batch.shape

torch.Size([2, 6, 3])

In [ ]:
ctx

tensor([[0.1668],
        [0.1534],
        [0.1537],
        [0.1479],
        [0.1582],
        [0.1451]], grad_fn=<MmBackward0>)

In [ ]:
dim = ctx.shape[0]

In [ ]:
mask = torch.triu(torch.ones(dim,dim), diagonal=1)

In [ ]:
mask

tensor([[0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])

In [ ]:
mask.bool()[:6,:6]

tensor([[False,  True,  True,  True,  True,  True],
        [False, False,  True,  True,  True,  True],
        [False, False, False,  True,  True,  True],
        [False, False, False, False,  True,  True],
        [False, False, False, False, False,  True],
        [False, False, False, False, False, False]])

In [ ]:
temp.masked_fill(mask.bool()[:5,:5], -torch.inf)

tensor([[0.2173,   -inf,   -inf,   -inf,   -inf],
        [0.1758, 0.1822,   -inf,   -inf,   -inf],
        [0.1742, 0.1822, 0.1818,   -inf,   -inf],
        [0.1696, 0.1749, 0.1744, 0.1593,   -inf],
        [0.1432, 0.1744, 0.1773, 0.1520, 0.2220]],
       grad_fn=<MaskedFillBackward0>)

In [ ]:
batch_num, row, feature_dim = batch.shape
out_dim = 4

In [ ]:
wq = nn.Linear(feature_dim,out_dim,bias=False)
wk = nn.Linear(feature_dim,out_dim,bias=False)
wv = nn.Linear(feature_dim,out_dim,bias=False)

In [ ]:
output_projection = nn.Linear(feature_dim,out_dim)

In [ ]:
context_length = 6

In [ ]:
torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)

tensor([[0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])

In [ ]:
keys = wq(batch)
queries = wq(batch)
values = wv(batch)

In [ ]:
queries

tensor([[[-0.4821,  0.4336,  0.0376,  0.2466],
         [-0.5965,  0.6734,  0.2635,  0.1029],
         [-0.5914,  0.6726,  0.2422,  0.1129],
         [-0.3124,  0.3564,  0.2207, -0.0035],
         [-0.3341,  0.4689, -0.2114,  0.2630],
         [-0.3838,  0.3892,  0.4338, -0.0944]],

        [[-0.4821,  0.4336,  0.0376,  0.2466],
         [-0.5965,  0.6734,  0.2635,  0.1029],
         [-0.5914,  0.6726,  0.2422,  0.1129],
         [-0.3124,  0.3564,  0.2207, -0.0035],
         [-0.3341,  0.4689, -0.2114,  0.2630],
         [-0.3838,  0.3892,  0.4338, -0.0944]]], grad_fn=<UnsafeViewBackward0>)

In [ ]:
keys

tensor([[[-0.4821,  0.4336,  0.0376,  0.2466],
         [-0.5965,  0.6734,  0.2635,  0.1029],
         [-0.5914,  0.6726,  0.2422,  0.1129],
         [-0.3124,  0.3564,  0.2207, -0.0035],
         [-0.3341,  0.4689, -0.2114,  0.2630],
         [-0.3838,  0.3892,  0.4338, -0.0944]],

        [[-0.4821,  0.4336,  0.0376,  0.2466],
         [-0.5965,  0.6734,  0.2635,  0.1029],
         [-0.5914,  0.6726,  0.2422,  0.1129],
         [-0.3124,  0.3564,  0.2207, -0.0035],
         [-0.3341,  0.4689, -0.2114,  0.2630],
         [-0.3838,  0.3892,  0.4338, -0.0944]]], grad_fn=<UnsafeViewBackward0>)

In [ ]:
keys.transpose(1,2)

tensor([[[-0.4821, -0.5965, -0.5914, -0.3124, -0.3341, -0.3838],
         [ 0.4336,  0.6734,  0.6726,  0.3564,  0.4689,  0.3892],
         [ 0.0376,  0.2635,  0.2422,  0.2207, -0.2114,  0.4338],
         [ 0.2466,  0.1029,  0.1129, -0.0035,  0.2630, -0.0944]],

        [[-0.4821, -0.5965, -0.5914, -0.3124, -0.3341, -0.3838],
         [ 0.4336,  0.6734,  0.6726,  0.3564,  0.4689,  0.3892],
         [ 0.0376,  0.2635,  0.2422,  0.2207, -0.2114,  0.4338],
         [ 0.2466,  0.1029,  0.1129, -0.0035,  0.2630, -0.0944]]],
       grad_fn=<TransposeBackward0>)

In [ ]:
attn_scores = queries @ keys.transpose(1, 2)

In [ ]:
attn_scores.shape

torch.Size([2, 6, 6])

In [ ]:
keys.view(batch_num,row,2,2)

tensor([[[[-0.4821,  0.4336],
          [ 0.0376,  0.2466]],

         [[-0.5965,  0.6734],
          [ 0.2635,  0.1029]],

         [[-0.5914,  0.6726],
          [ 0.2422,  0.1129]],

         [[-0.3124,  0.3564],
          [ 0.2207, -0.0035]],

         [[-0.3341,  0.4689],
          [-0.2114,  0.2630]],

         [[-0.3838,  0.3892],
          [ 0.4338, -0.0944]]],


        [[[-0.4821,  0.4336],
          [ 0.0376,  0.2466]],

         [[-0.5965,  0.6734],
          [ 0.2635,  0.1029]],

         [[-0.5914,  0.6726],
          [ 0.2422,  0.1129]],

         [[-0.3124,  0.3564],
          [ 0.2207, -0.0035]],

         [[-0.3341,  0.4689],
          [-0.2114,  0.2630]],

         [[-0.3838,  0.3892],
          [ 0.4338, -0.0944]]]], grad_fn=<ViewBackward0>)

In [ ]:
# # so i finally figured out the complexity of this multi head attention and how each
# head is able to generate differnet meaning along with why we need to have the
# output dim = the heads
# so what is happening is that when we decide on this output dim which is technically
# a hyper parameter for the linear projections we inherently determine the output dim
# now what happens is that in single head attention we get the context vector or
# the dim of our choosing but when working with mha we work on the slices of these
# token embeddings which is why each of the head in mha is able to generate differnet
# output for the same input. its like defining 'i hate you' in a 768 dim vector and then
# projection to the linear layer to get say 1024 dim and finally when we reach the mha
# it works on all of the tokens but on the slices of the features. this is why we need
# the head to div the output dim as we are splitting the embeddings into these heads.
# In single-head attention, output dimension is unconstrained.
# In multi-head attention, the chosen output dimension must be divisible by the
# number of heads so it can be evenly split across them.

# "love":
# Head 1 → [f1, f2]
# Head 2 → [f3, f4]

In [ ]:
import torch.nn as nn
import torch
class CausalAttention(nn.Module):
  def __init__(self, id, od, context_win, bias=False):
    super().__init__()
    self.qw = nn.Linear(id,od,bias=bias)
    self.qk = nn.Linear(id,od,bias=bias)
    self.qv = nn.Linear(id,od,bias=bias)
    self.register_buffer('mask',torch.triu(torch.ones(context_win,context_win),diagonal=1))

  def forward(self,x):
    b, context_win, dim_out =  x.shape
    # here the contextwin is safety measure if we ever exceed the window which is in the batch
    self.K = self.qk(x)
    self.Q = self.qw(x)
    self.V = self.qv(x)

    attn_scores = self.K @ self.Q.transpose(1,2)
    attn_scores.masked_fill_(self.mask.bool()[:context_win,:context_win],-torch.inf)
    attn_weights = torch.softmax(attn_scores / self.K.shape[0]**0.5,dim=-1)
    context_vec = attn_weights @ self.V

    return context_vec

In [ ]:
# this here is for just a single two dim tensor with no batch
import torch.nn as nn
import torch
class CausalAttention(nn.Module):
  def __init__(self, id, od, context_win, bias=False):
    super().__init__()
    self.qw = nn.Linear(id,od,bias=bias)
    self.qk = nn.Linear(id,od,bias=bias)
    self.qv = nn.Linear(id,od,bias=bias)
    self.register_buffer('mask',torch.triu(torch.ones(context_win,context_win),diagonal=1))

  def forward(self,x):
    context_win, dim_out =  x.shape
    # here the contextwin is safety measure if we ever exceed the window which is in the batch
    self.K = self.qk(x)
    self.Q = self.qw(x)
    self.V = self.qv(x)

    attn_scores = self.K @ self.Q.T
    attn_scores.masked_fill_(self.mask.bool()[:context_win,:context_win],-torch.inf)
    attn_weights = torch.softmax(attn_scores / self.K.shape[0]**0.5,dim=-1)
    context_vec = attn_weights @ self.V

    return context_vec

In [ ]:
batch

tensor([[[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]],

        [[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]]])

In [ ]:
batch_size, rows, features = batch.shape
print(batch_size, rows, features)

2 6 3


In [ ]:
hyperparamter = 4

In [ ]:
inputs

tensor([[0.4300, 0.1500, 0.8900],
        [0.5500, 0.8700, 0.6600],
        [0.5700, 0.8500, 0.6400],
        [0.2200, 0.5800, 0.3300],
        [0.7700, 0.2500, 0.1000],
        [0.0500, 0.8000, 0.5500]])

In [ ]:

# features are as we know the dim of input, hyperparametes here is of choosing and rows is the no of tokens you have, we need this cause when
# doing causal masking we mask over tokens so we need to know the no of token inorder to make tensor for our masking.
model = CausalAttention(3,hyperparamter,6)
attn = model(inputs)

In [ ]:
attn

tensor([[ 0.3423, -0.0886, -0.1723, -0.2687],
        [ 0.3337, -0.0659, -0.0720, -0.2820],
        [ 0.3267, -0.0631, -0.0420, -0.2772],
        [ 0.2912, -0.0412, -0.0169, -0.2626],
        [ 0.2305, -0.0922, -0.0348, -0.1460],
        [ 0.2547, -0.0402, -0.0028, -0.2251]], grad_fn=<MmBackward0>)

In [ ]:
import torch.nn as nn
import torch

class MultiHeadAttention(nn.Module):


  def __init__(self,idim,odim,context_length,num_heads,bias=False):
    super().__init__()
    self.odim = odim
    self.num_head = num_heads
    self.head_dim = odim // self.num_head

    self.W_Q = nn.Linear(idim,odim,bias=False)
    self.W_K = nn.Linear(idim,odim,bias=False)
    self.W_V = nn.Linear(idim,odim,bias=False)

    self.register_buffer('mask',torch.triu(torch.ones(context_length,context_length),diagonal=1))
    self.out_proj = nn.Linear(odim,odim)

  def forward(self,x):
    batch, num_row, feature_dim = x.shape

    self.K = self.W_K(x)
    self.Q = self.W_Q(x)
    self.V = self.W_V(x)

    # crucial step for mha, we are using non overlapping embedding dim for each of the token
    self.K = self.K.view(batch,num_row,self.num_head,self.head_dim)
    self.Q = self.Q.view(batch,num_row,self.num_head,self.head_dim)
    self.V = self.V.view(batch,num_row,self.num_head,self.head_dim)

    self.K = self.K.transpose(1,2)
    self.Q = self.Q.transpose(1,2)
    self.V = self.V.transpose(1,2)

    # attn_score would always be of the num row length since we are working with batches we need all the tokens which come from rows
    attn_score = self.K @ self.Q.transpose(2,3)
    attn_score.masked_fill_(self.mask.bool()[:num_row,:num_row],-torch.inf)
    attn_weight = torch.softmax(attn_score / self.K.shape[-1]**0.5, dim=-1)
    context_vector = attn_weight @ self.V
    context_vector = context_vector.contiguous().view(batch,num_row,self.odim)
    # this out_proj is just here to get genrate these nuance in your multiple context vector so its kinda linear transformation of weights
    # to combine the informatino across the heads you need to perform this operation so the heads can talk to each other.
    context_vector = self.out_proj(context_vector)

    # out_projection = self.out_proj(batch,num_row,self.odim)
    return context_vector

In [ ]:
batch.shape

torch.Size([2, 6, 3])

In [ ]:
idim = batch.shape[-1]
hyperparameter = 4
odim = hyperparameter
context_length = batch.shape[1]
num_heads = 2

In [ ]:
model = MultiHeadAttention(idim, odim, context_length, num_heads)

In [ ]:
model(batch)

tensor([[[-0.0356, -0.3241,  0.3343, -0.1556],
         [ 0.0788, -0.3076,  0.4104, -0.2109],
         [ 0.0801, -0.3135,  0.4274, -0.2323],
         [-0.0978, -0.5183,  0.2747, -0.4525],
         [-0.1148, -0.5278,  0.3900, -0.5336],
         [-0.1275, -0.5350,  0.3865, -0.5371]],

        [[-0.0356, -0.3241,  0.3343, -0.1556],
         [ 0.0788, -0.3076,  0.4104, -0.2109],
         [ 0.0801, -0.3135,  0.4274, -0.2323],
         [-0.0978, -0.5183,  0.2747, -0.4525],
         [-0.1148, -0.5278,  0.3900, -0.5336],
         [-0.1275, -0.5350,  0.3865, -0.5371]]], grad_fn=<ViewBackward0>)